<a href="https://colab.research.google.com/github/khanhquynh25/DemoGit/blob/master/m%C3%B3n_%C4%83n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from google.colab import drive
import gradio as gr
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array

# ================== 1. KẾT NỐI GOOGLE DRIVE ==================
drive.mount("/content/drive")

# Thư mục chứa dataset trong Drive
# Cấu trúc thư mục phải như sau:
# /content/drive/MyDrive/monan/
#    ├── pho/
#    ├── com/
#    ├── rau/
#    ├── thit/
#    └── trung/
base_dir = "/content/drive/MyDrive/monan"

# ================== 2. TIỀN XỬ LÝ DỮ LIỆU ==================
img_size = (128, 128)
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.3  # 30% dữ liệu cho validation
)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    subset="training"
)

val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    subset="validation"
)

# Lấy nhãn lớp
labels = list(train_generator.class_indices.keys())
print("📌 Các lớp:", labels)

# ================== 3. XÂY DỰNG MÔ HÌNH ANN ==================
model = Sequential([
    Flatten(input_shape=(img_size[0], img_size[1], 3)),
    Dense(512, activation="relu"),
    Dropout(0.4),
    Dense(256, activation="relu"),
    Dropout(0.3),
    Dense(len(labels), activation="softmax")
])

# ================== 4. COMPILE ==================
model.compile(optimizer=Adam(learning_rate=0.0005),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# ================== 5. TRAIN ==================
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30
)

# ================== 6. LƯU MÔ HÌNH VÀO DRIVE ==================
save_path = "/content/drive/MyDrive/monan/food_ann.h5"
model.save(save_path)
print(f"✅ Đã train xong và lưu mô hình ANN tại: {save_path}")

# ================== 7. GIAO DIỆN GRADIO ==================

def predict_and_show(img):
    img_resized = img.resize(img_size)   # resize về (128,128)
    img_array = img_to_array(img_resized) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    preds = model.predict(img_array)
    class_idx = np.argmax(preds)
    confidence = np.max(preds)
    label = labels[class_idx]

    return f"🍽️ Đây là món: {label}\n(Độ tin cậy: {confidence:.2f})"

# ================== CSS giao diện ==================
custom_css = """
.gradio-container {background-color: #fff5e6; font-family: Arial;}
h1, h2, h3 {color: #cc6600; font-weight: bold;}
label {color: #994d00; font-weight: bold;}
button {background-color: #ff6600 !important; color: white !important;
        font-weight: bold; border-radius: 10px !important; padding: 10px 20px !important;}
.output-textbox textarea {font-size: 22px !important; font-weight: bold !important; color: #cc3300;}
"""

demo = gr.Interface(
    fn=predict_and_show,
    inputs=gr.Image(type="pil", label="📷 Upload ảnh món ăn"),
    outputs=gr.Textbox(label="📌 Kết quả dự đoán"),
    title="🍽️ Nhận diện món ăn Việt",
    description="Upload ảnh phở, cơm, rau, thịt hoặc trứng để nhận diện.",
    css=custom_css
)

demo.launch(debug=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 21 images belonging to 5 classes.
Found 6 images belonging to 5 classes.
📌 Các lớp: ['com', 'pho', 'rau', 'thit', 'trung']


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - accuracy: 0.1905 - loss: 2.2343 - val_accuracy: 0.3333 - val_loss: 6.7578
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.1429 - loss: 9.5995 - val_accuracy: 0.1667 - val_loss: 18.0616
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.3333 - loss: 18.3504 - val_accuracy: 0.1667 - val_loss: 16.7334
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.3810 - loss: 15.8688 - val_accuracy: 0.1667 - val_loss: 11.0511
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 801ms/step - accuracy: 0.1429 - loss: 17.0471 - val_accuracy: 0.1667 - val_loss: 8.0219
Epoch 6/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 804ms/step - accuracy: 0.1429 - loss: 14.1513 - val_accuracy: 0.5000 - val_loss: 6.3520
Epoch 7/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 813ms/step - accuracy: 0.4762 - loss: 14.6301 - val_accuracy: 0.3333 - val_loss: 9.1326
Epoch 8/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 844ms/step - accuracy: 0.4286 - loss: 10.2286 - val_accuracy: 0.5000 - val_loss: 6.

✅ Đã train xong và lưu mô hình ANN tại: /content/drive/MyDrive/monan/food_ann.h5
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f498a35e905ad452ba.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
